In [1]:
import pandas as pd
import os
from pathlib import Path
import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split, KFold
from PIL import Image
from ImageAnalysis import ImageAnalysis
from ImageTableExtraction import ImageTableExtraction
from SudokuDetection import SudokuDetection
from solvers.sudoku.sudoku_solver import SudokuSolver

In [230]:
mpl.rcParams['figure.dpi']= 200
plt.rcParams['figure.figsize'] = [5,4]

In [231]:
data_folder = "../data/original/"
annotated_folder = "../data/training_data/"
testdata_folder = "../data/testdata_dont_touch/"

In [232]:
def read_samples(folder):
    samples = []
    for file in os.listdir(data_folder):
         filename = os.fsdecode(file)
         if filename.endswith(".jpg"):
            fullname = data_folder + file
            fullname_no_extension = data_folder + Path(fullname).stem
            outcome = pd.read_json(fullname_no_extension + ".json")
            samples.append((filename, outcome))
    return samples

In [233]:
samples = read_samples(data_folder)

In [234]:
def eval(samples):
    success = [x for x in samples if len(x[1]["solution"]) > 0]
    print(len(success))
    print(len(samples))
    return len(success) / len(samples)

In [235]:
eval(samples)

63
217


0.2903225806451613

In [236]:
X_train, X_test, y_train, y_test = train_test_split(samples, samples, test_size=0.20, random_state=42)
print(len(X_train))
print(len(X_test))

173
44


In [237]:
def move_annotated_files(samples, source_folder, destination_folder):
    for test_sample in X_test:
        image_filepath = test_sample[0]
        image_fullpath = source_folder + image_filepath
        image_target = destination_folder + image_filepath
        name_no_ext = Path(image_filepath).stem
        annotation_filepath = name_no_ext + ".xml"
        annotation_fullpath = source_folder + annotation_filepath
        annotation_target = destination_folder + annotation_filepath
        filename = os.fsdecode(image_filepath)
        os.rename(image_fullpath, image_target)
        os.rename(annotation_fullpath, annotation_target)

In [238]:
class SudolverModel():
    def __init__(self):
        self.extractor = None
        self.persp_transform = False
        self.filters = []
        self.solver = SudokuSolver()
        self.detector = SudokuDetection(ImageAnalysis(), ImageTableExtraction())
        
    def predict(self, image_rgb):
        try:
            result = image_rgb
            if self.extractor is not None:
                preprocessed = self.extractor.preprocess(result)
                result = self.extractor.predict(preprocessed, self.persp_transform)
                if result is None:
                    print("YOLO didn't detect sudoku")
                    return 0
            for f in self.filters:
                result = f.apply_filter(result)
            return self.scan_and_solve(result)
        except Exception as ex:
            print(ex)
            return 0
    
    def get_params(self):
        desc = ""
        if self.extractor is not None:
            desc += "Extractor.Activated,"
        for f in self.filters:
            desc += f.get_params() + ","
        return desc
    
    def enable_extractor(self):
        self.extractor = SudokuExtractor()
    
    def add_filter(self, new_filter):
        self.filters.append(new_filter)
    
    def scan_and_solve(self, image_rgb):
        result = None
        try:
            success, encoded_image = cv2.imencode('.jpg', image_rgb)
            content = encoded_image.tobytes()
            success, result = self.detector.detect(content)
            if success:
                solution = self.solver.solve(result)
                return 1
            else:
                print(result)
                return 0
        except Exception as ex:
            print(ex)
            if result is not None:
                print(result)
            return 0

In [239]:
from tqdm import tqdm
def eval_all(samples):
    extractors = (None, SudokuExtractor())
    sharp_variants = (None, Laplacian(10))
    results = []
    total = len(samples)
    
    for enable_extractor in extractors:
        for sharp in sharp_variants:
                sudolver = SudolverModel()
                if enable_extractor is not None:
                    sudolver.enable_extractor()
                if sharp is not None:
                    sudolver.add_filter(sharp)
                model_total = 0
                for sample in tqdm(samples):
                    filename = sample[0]
                    image = cv2.imread(data_folder + filename)
                    image_rgb = to_rgb_from_bgr(image)
                    success = sudolver.predict(image_rgb)
                    model_total += success
                model_desc = sudolver.get_params()
                eval_result = (model_desc, model_total / total)
                print(eval_result)
                results.append(eval_result)
    return results

In [240]:
def read_image(filepath):
    return cv2.imread(filepath)

In [241]:
def to_rgb_from_bgr(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [242]:
def to_bgr_from_rgb(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

In [243]:
def to_grayscale_from_bgr(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [244]:
def eq_hist_from_bgr(image):
    return cv2.equalizeHist(to_grayscale_from_bgr(image))

In [245]:
def unsharp_masking(image, k=10):
    normalized = image / 255.0
    blurred = cv2.GaussianBlur(normalized, [5, 5], 3, 3)
    mask = normalized - blurred
    sharp = normalized + k*mask
    sharp[sharp > 1] = 1
    sharp[sharp < 0] = 0
    sharp = sharp * 255
    sharp = sharp.astype(np.uint8)
    return sharp

In [246]:
def laplacian(image, alpha=10):
    normalized = image / 255.0
    laplacian = cv2.Laplacian(normalized, cv2.CV_64F)
    sharp = normalized - alpha * laplacian
    sharp[sharp > 1] = 1
    sharp[sharp < 0] = 0
    sharp = sharp * 255
    sharp = sharp.astype(np.uint8)
    return sharp

In [247]:
filepath = annotated_folder + X_train[34][0]

In [248]:
image = read_image(filepath)
#plt.imshow(to_rgb_from_bgr(image))

In [249]:
from imgaug.augmenters.meta import Sequential
import imgaug.augmenters as iaa
import imageio
import torch

class SudokuExtractor():
    def __init__(self):
        self.model = torch.hub.load('C:/Projects/sudolver-models/data/model/yolov5', 'custom', path='C:/Projects/sudolver-models/data/trained_model/best_2022-04-09.pt', source='local') 

    def preprocess(self, image):
        aug = [iaa.CenterPadToSquare(), iaa.Resize({"height": 640, "width": 640})]
        seq = iaa.Sequential(aug)
        return seq(image=image)
        
    def predict(self, image, persp_transform):
        result = self.model(image)
        if len(result) > 0:
            pred = result.pandas()
            xmin, ymin, xmax, ymax = (int(pred.xyxy[0]['xmin'].values[0]), int(pred.xyxy[0]['ymin'].values[0]), int(pred.xyxy[0]['xmax'].values[0]), int(pred.xyxy[0]['ymax'].values[0]))
            img = to_rgb_from_bgr(perspective_transform(image, self.pred_to_conjugates(xmin, ymin, xmax, ymax)))
            return img
        else:
            return None
        
    def pred_to_conjugates(self, xmin, ymin, xmax, ymax):
        return np.float32([[xmin, ymin], [xmax, ymin], [xmax, ymax], [xmin, ymax]])

In [250]:
class ImageFilter():
    def apply_filter(self, image):
        pass
    
    def get_params(self):
        pass
    
class UnsharpMasking(ImageFilter):
    def __init__(self, k=5):
        self.k = k
        
    def apply_filter(self, image_rgb):
        return unsharp_masking(image_rgb, self.k)
    
    def get_params(self):
        return f"UnsharpMasking (k={self.k})"
    
class Laplacian(ImageFilter):
    def __init__(self, alpha=5):
        self.alpha = alpha
        
    def apply_filter(self, image_rgb):
        return laplacian(image_rgb, self.alpha)
    
    def get_params(self):
        return f"Laplacian (alpha={self.alpha})"
    
class HigherDPI(ImageFilter):
    def __init__(self, dpi=150):
        self.dpi = dpi
        
    def apply_filter(self, image_rgb):
        im = Image.fromarray(image_rgb)
        temp_filename = "C:/Projects/sudolver-models/data/high_dpi/temp.jpg"
        im.save(temp_filename, dpi=(self.dpi, self.dpi))
        return to_rgb_from_bgr(read_image(temp_filename))
    
    def get_params(self):
        return f"Higher DPI = {self.dpi}"

In [251]:
results = eval_all(X_test)
print(results)

requirements: pandas>=1.1.4 not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'pandas>=1.1.4'' returned non-zero exit status 1.
YOLOv5  v6.1-118-g3bb233a torch 1.10.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
  2%|█▉                                                                                 | 1/44 [00:01<01:01,  1.43s/it]

[[['5', '7', '', '', '8', '', '3', '2 1'], ['', '', '', '', '5', '2', '', ''], ['', '5', '6', '3', '', '', '', ''], ['', '', '1', '', '2', '', '7', ''], ['', '', '', '', '', '5', '9', '6']]]


  5%|███▊                                                                               | 2/44 [00:03<01:02,  1.49s/it]

[[['4', '', '', '', '', '9', '8', '', '6'], ['', '', '5', '', '3', '', '', '', '7'], ['3', '', '6', '8', '', '7', '1', '', ''], ['5', '', '', '', '', '1', '6', '8', ''], ['', '7', '9', '', '', '', '4', '2', ''], ['', '2', '8', '4', '', '', '', '', '3'], ['', '', '3', '6', '', '4', '2', '', '1']]]


 11%|█████████▍                                                                         | 5/44 [00:08<01:03,  1.62s/it]

...

[[['', '', '', '', '4', '', '', '', '1'], ['', '5', '8', '', '9', '', '', '', '3'], ['1', '4', '', '7', '', '3', '2', '6', ''], ['', '', '6', '', '8', '', '', '3', ''], ['', '1', '', '4', '', '2', '6', '', '9'], ['3', '7', '', '9', '', '', '1', '', '2'], ['', '3', '', '8', '', '6', '', '9', '7'], ['', '', '', '', '2', '9', '', '1', ''], ['9', '6', 'n', '', '', '', '', '', '']]]


 98%|████████████████████████████████████████████████████████████████████████████████▏ | 43/44 [01:24<00:01,  1.89s/it]

[[['', '8', '', '', '', '5 4'], ['', '', '5', '', '2', '1'], ['', '', '7', '4', '', '3'], ['', '', '', '', '', '7 4'], ['6', '', '4', '', '1', '8 9'], ['', '', '3', '6', '', ''], ['', '', '9', '', '', '4 5'], ['', '5', '', '', '6', '7'], ['', '6', '', '9', '', '2 4']]]


100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [01:27<00:00,  1.98s/it]

('Extractor.Activated,Laplacian (alpha=10),', 0.6590909090909091)
[('', 0.2727272727272727), ('Laplacian (alpha=10),', 0.4772727272727273), ('Extractor.Activated,', 0.1590909090909091), ('Extractor.Activated,Laplacian (alpha=10),', 0.6590909090909091)]
